<a href="https://colab.research.google.com/github/IraJasper/DuLabCode/blob/main/NV_measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import TimeTagger
import numpy as np
import time
from scipy.signal import butter, cheby1, filtfilt
import matplotlib.pyplot as plt
from hardware.timetagger_counter import HWRecorderMode
import itertools
from scipy.optimize import curve_fit
from datetime import date
from tqdm import tqdm
import os

In [ ]:
def get_data(analyze_window_start,analyze_window_length):

    counts_raw = counter.getData()
    counts = np.sum(counts_raw[:,analyze_window_start:analyze_window_start+analyze_window_length],1)

    return counts

In [ ]:
def AWG_waveform(block): # convert [(time, voltage)] sequence to AWG sequence
    AWG_sampling_rate = 1.2e9 # AWG5014C has sampling rate 1.2GS/s

    sampled_wave=[]

    for elements in block:
        num_sample_points = round(elements[0]*1e-9*AWG_sampling_rate)
        sampled_list=[elements[1]] * num_sample_points
        sampled_wave.extend(sampled_list)

    sampled_wave = np.array(sampled_wave)

    return sampled_wave

In [ ]:
class MeasurementDefinitions:
    def pulsed_ODMR(self, parameters):
        pi_pulse = parameters['pi_pulse']
        freq_dev = parameters['freq_dev']
        num_freq = parameters['num_freq']

        laser_length = 1500 #ns
        wait_length = 400 #ns
        analog_delay = 000 #ns
        pulsed_length = pi_pulse+laser_length+wait_length #ns

        mw_mod_length = pulsed_length*1 #ns
        meas_time  = 0

        binwidth = 1 # ns
        binwidth_ps = binwidth*1000 #ps
        n_bins = laser_length/binwidth

        mw_mod_output = np.linspace(-1*freq_dev/4e7,1*freq_dev/4e7,num_freq)

        block_mw_mod = []
        block_mw_switch = []
        block_laser = []
        block_gate = []

        block_mw_switch.extend([(analog_delay,0)])
        block_laser.extend([(analog_delay,0)])
        block_gate.extend([(analog_delay,0)])
        for i in range(num_freq):
            block_mw_mod.extend([(mw_mod_length,mw_mod_output[i])])
            block_mw_switch.extend([(pi_pulse, 1), (laser_length, 0), (wait_length, 0)]) #2-mw switch
            block_laser.extend([(pi_pulse, 0), (laser_length, 1), (wait_length, 0)]) #0-laser
            block_gate.extend([(pi_pulse, 0),(laser_length, 1), (wait_length, 0)]) #5-TT start
            meas_time+= pi_pulse+laser_length+wait_length

        block_mw_mod.extend([(analog_delay,0)])
        meas_time+= analog_delay

        sequences = {}
        # Use names instead of (type, channel)
        sequences['mw_mod'] = block_mw_mod
        sequences['mw_switch'] = block_mw_switch
        sequences['laser'] = block_laser
        sequences['gate'] = block_gate
        sequences['sync'] = [(meas_time,0),(40,1)]

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': int(n_bins),
            'n_histograms': int(num_freq)
        }

        return sequences, tagger_config, meas_time

    def CW_ODMR(self, parameters):
        pi_pulse = parameters['pi_pulse']
        freq_dev = parameters['freq_dev']
        num_freq = parameters['num_freq']

        laser_length = 1500 #ns
        wait_length = 400 #ns
        analog_delay = 000 #ns
        pulsed_length =laser_length+wait_length #ns,CW

        mw_mod_length = pulsed_length*1 #ns
        meas_time  = 0

        binwidth = 1 # ns
        binwidth_ps = binwidth*1000 #ps
        n_bins = laser_length/binwidth

        mw_mod_output = np.linspace(-1*freq_dev/4e7,1*freq_dev/4e7,num_freq)

        block_mw_mod = []
        block_mw_switch = []
        block_laser = []
        block_gate = []

        block_mw_switch.extend([(analog_delay,0)])
        block_laser.extend([(analog_delay,0)])
        block_gate.extend([(analog_delay,0)])
        for i in range(num_freq):
            block_mw_mod.extend([(mw_mod_length,mw_mod_output[i])])
            block_mw_switch.extend([ (laser_length, 1), (wait_length, 0)]) #2-mw switch
            block_laser.extend([(laser_length, 1), (wait_length, 0)]) #0-laser
            block_gate.extend([(laser_length, 1), (wait_length, 0)]) #5-TT start
            meas_time+= laser_length+wait_length

        block_mw_mod.extend([(analog_delay,0)])
        meas_time+= analog_delay

        sequences = {}
        sequences['mw_mod'] = block_mw_mod
        sequences['mw_switch'] = block_mw_switch
        sequences['laser'] = block_laser
        sequences['gate'] = block_gate
        sequences['sync'] = [(meas_time,0),(40,1)]

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': int(n_bins),
            'n_histograms': int(num_freq)
        }

        return sequences, tagger_config, meas_time

    def rabi(self,parameters):
        laser_time = parameters['laser_time']
        wait_time = parameters['wait_time']
        tau_start = parameters['tau_start']
        tau_end = parameters['tau_end']
        num_points = parameters['num_points']

        taus = np.linspace(tau_start,tau_end,num_points)

        block_laser = []
        block_counter = []
        block_mw = []

        meas_time = 0

        for tau in taus:
            block_laser.extend([(tau,0),(laser_time,1),(wait_time,0)])
            block_counter.extend([(tau,0),(laser_time,1),(wait_time,0)])
            block_mw.extend([(tau,1),(laser_time,0),(wait_time,0)])
            meas_time += laser_time+wait_time+tau

        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40

        sequences = {}
        sequences['laser'] = block_laser
        sequences['counter'] = block_counter
        sequences['mw'] = block_mw
        sequences['sync'] = block_sync

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': laser_time,
            'n_histograms': num_points
        }

        return sequences, tagger_config, meas_time

    def rabi_IQ(self,parameters):
        laser_time = parameters['laser_time']
        wait_time = parameters['wait_time']
        tau_start = parameters['tau_start']
        tau_end = parameters['tau_end']
        num_points = parameters['num_points']
        phi = parameters['phi']

        taus = np.linspace(tau_start,tau_end,num_points)

        block_laser = []
        block_counter = []
        block_I = []
        block_Q = []

        meas_time = 0

        for tau in taus:
            block_laser.extend([(tau,0),(laser_time,1),(wait_time,0)])
            block_counter.extend([(tau,0),(laser_time,1),(wait_time,0)])
            block_I.extend([(tau,0.5*np.sin(phi)),(laser_time,0),(wait_time,0)])
            block_Q.extend([(tau,0.5*np.cos(phi)),(laser_time,0),(wait_time,0)])
            meas_time += laser_time+wait_time+tau

        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40

        sequences = {}
        sequences['laser'] = block_laser
        sequences['counter'] = block_counter
        sequences['I'] = block_I
        sequences['Q'] = block_Q
        sequences['sync'] = block_sync

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': laser_time,
            'n_histograms': num_points
        }

        return sequences, tagger_config, meas_time

    def T1(self,parameters):
        num_points = parameters['num_points']
        start_time = parameters['start_time'] #ns
        stop_time = parameters['stop_time'] #ns
        delay_time = np.geomspace(start_time, stop_time, num_points) #ns
        pi_pulse = parameters['pi_pulse'] #ns
        laser_time = parameters['laser_time'] # 1.6e3 #ns
        wait_time = parameters['wait_time'] # 500 #ns

        meas_time = 0
        block_laser = []

        block_mw = []

        for i in range(num_points):
            time_trace = [delay_time[i],pi_pulse,laser_time,wait_time,
                          delay_time[i],pi_pulse,laser_time,wait_time]
            laser_on_off = [0,0,1,0,0,0,1,0]
            mw_on_off = [0,0,0,0,0,1,0,0,]
            block_laser.extend(list(zip(time_trace,laser_on_off)))
            block_mw.extend(list(zip(time_trace,mw_on_off)))
            meas_time += sum(time_trace)

        block_counter = block_laser
        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40
        sequences = {}
        sequences['laser'] = block_laser
        sequences['counter'] = block_counter
        sequences['mw'] = block_mw
        sequences['sync'] = block_sync

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': laser_time,
            'n_histograms': num_points*2
        }

        return sequences, tagger_config, meas_time

    def T1_SCC(self,parameters):
        num_points = parameters['num_points']
        start_time = parameters['start_time'] #ns
        stop_time = parameters['stop_time'] #ns
        delay_time = np.geomspace(start_time, stop_time, num_points) #ns

        pi_pulse = parameters['pi_pulse'] #ns

        # config
        G_inti_time = parameters['G_inti_time']
        G_delay = parameters['G_delay']
        Y_read_time = parameters['Y_read_time']
        Y_delay = parameters['Y_delay']
        R_ionize_time = parameters['R_ionize_time']
        R_delay = parameters['R_delay']

        meas_time = 0

        block_G = []
        block_Y = []
        block_R = []
        block_mw = []
        block_counter = []
        for i in range(num_points):
            time_trace = [G_inti_time,G_delay,delay_time[i],pi_pulse,
                          R_ionize_time,R_delay,Y_read_time,Y_delay,
                          G_inti_time,G_delay,delay_time[i],pi_pulse,
                          R_ionize_time,R_delay,Y_read_time,Y_delay]
            G_on_off = [1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
            Y_on_off = [0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0]
            R_on_off = [0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0]
            mw_on_off = [0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0]
            counter_on_off = [0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1] # Due to the delay of yellow laser, the counter should be on during Y_delay to read the PL
            block_counter.extend(list(zip(time_trace,counter_on_off)))
            block_G.extend(list(zip(time_trace,G_on_off)))
            block_Y.extend(list(zip(time_trace,Y_on_off)))
            block_R.extend(list(zip(time_trace,R_on_off)))
            block_mw.extend(list(zip(time_trace,mw_on_off)))
            meas_time += sum(time_trace)

        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40

        sequences = {}
        sequences['G'] = block_G
        sequences['Y'] = block_Y
        sequences['R'] = block_R
        sequences['mw'] = block_mw
        sequences['sync'] = block_sync
        sequences['counter'] = block_counter

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e5,
            'n_bins': (Y_read_time+Y_delay)/1e2,
            'n_histograms': num_points*2
        }

        return sequences, tagger_config, meas_time

    def T2(self,parameters):
        xy_num = parameters['xy_num']
        xy_repetition = parameters['xy_repetition']
        start_phase = parameters['start_phase']
        end_phase = parameters['end_phase']
        laser_time = parameters['laser_time']
        wait_time = parameters['wait_time']
        mw_wait = parameters['mw_wait']
        pi_pulse = parameters['pi_pulse']
        num_points = parameters['num_points']
        tau_start = parameters['tau_start']
        tau_end = parameters['tau_end']
        taus = np.linspace(tau_start,tau_end,num_points)

        def xy_analog_on_off_one_block(i):
            if i % 2 == 0:
                A0_to_extend = [0,0.5,0]
                A1_to_extend = [0,0,0]
            elif i % 2 == 1:
                A0_to_extend = [0,0,0]
                A1_to_extend = [0,0.5,0]
            return A0_to_extend, A1_to_extend

        def xy_analog_on_off_four_block():
            A0_on_off_four = []
            A1_on_off_four = []
            for i in range(4):
                A0_to_extend,A1_to_extend = xy_analog_on_off_one_block(i)
                A0_on_off_four.extend(A0_to_extend)
                A1_on_off_four.extend(A1_to_extend)
            return A0_on_off_four,A1_on_off_four

        def xy_analog_on_off():

            if start_phase == 'X' or start_phase == 'x':
                phase = [0.5,0]
            elif start_phase == 'Y' or start_phase == 'y':
                phase = [0,0.5]
            elif start_phase == '-X' or start_phase == '-x':
                phase = [-0.5,0]
            elif start_phase == '-Y' or start_phase == '-y':
                phase = [0,-0.5]

            A0_on_off = [phase[0],0]
            A1_on_off = [phase[1],0]
            A0_on_off_four,A1_on_off_four = xy_analog_on_off_four_block()

            for _ in range(xy_repetition):
                if xy_num == 1:
                    A0_to_extend,A1_to_extend = xy_analog_on_off_one_block(0)
                    A0_on_off.extend(A0_to_extend)
                    A1_on_off.extend(A1_to_extend)
                elif xy_num == 4:
                    A0_on_off.extend(A0_on_off_four)
                    A1_on_off.extend(A1_on_off_four)
                elif xy_num == 8:
                    A0_on_off.extend(A0_on_off_four+A1_on_off_four)
                    A1_on_off.extend(A1_on_off_four+A0_on_off_four)
                elif xy_num == 16:
                    A0_on_off_four_conjugate = [-element for element in A0_on_off_four]
                    A1_on_off_four_conjugate = [-element for element in A1_on_off_four]
                    A0_on_off.extend(A0_on_off_four+A1_on_off_four+A0_on_off_four_conjugate+A1_on_off_four_conjugate)
                    A1_on_off.extend(A1_on_off_four+A0_on_off_four+A1_on_off_four_conjugate+A0_on_off_four_conjugate)

            if end_phase == 'X' or end_phase == 'x':
                phase = [0.5,0]
            elif end_phase == 'Y' or end_phase == 'y':
                phase = [0,0.5]
            elif end_phase == '-X' or end_phase == '-x':
                phase = [-0.5,0]
            elif end_phase == '-Y' or end_phase == '-y':
                phase = [0,-0.5]

            A0_on_off.extend([phase[0],0,0,0])
            A1_on_off.extend([phase[1],0,0,0])

            return A0_on_off,A1_on_off

        def time_trace(tau):
            time_trace = [pi_pulse/2,mw_wait]
            for _ in range(xy_repetition):
                for _ in range(xy_num):
                    time_trace.extend([tau/(2*xy_num*xy_repetition),pi_pulse,tau/(2*xy_num*xy_repetition)])
            time_trace.extend([pi_pulse/2,mw_wait,laser_time,wait_time])
            return time_trace

        def xy_block(tau):
            time_trace = time_trace(tau)
            I_block = []
            Q_block = []

            I_on_off,Q_on_off = xy_analog_on_off()
            for i in range(len(time_trace)):
                I_block.extend([(time_trace[i],I_on_off[i])])
                Q_block.extend([(time_trace[i],Q_on_off[i])])

            laser_block = [(time,0) for time in time_trace]
            laser_block[-2] = (laser_time,1)
            counter_block = laser_block

            return I_block,Q_block,laser_block,counter_block


        block_laser = []
        block_counter = []
        block_I = []
        block_Q = []
        meas_time = 0
        for tau in taus:
            I_block,Q_block,laser_block,counter_block = xy_block(tau)
            block_laser.extend(laser_block)
            block_counter.extend(counter_block)
            block_I.extend(I_block)
            block_Q.extend(Q_block)
            meas_time += laser_time+wait_time+tau

        block_sync = [(meas_time,0),(40,1)]
        meas_time += 40

        sequences = {}
        sequences['laser'] = block_laser
        sequences['counter'] = block_counter
        sequences['I'] = block_I
        sequences['Q'] = block_Q
        sequences['sync'] = block_sync

        tagger_config = {
            'click_channel': 1,
            'start_channel': 2,
            'next_channel': 2,
            'sync_channel': 3,
            'binwidth': 1e3,
            'n_bins': laser_time,
            'n_histograms': num_points
        }

        return sequences, tagger_config, meas_time

In [ ]:
class measurement_sequence:
    def __init__(self, parameters, channel_map, definitions_class=MeasurementDefinitions):
        self.parameters = parameters
        self.channel_map = channel_map
        self.definitions = definitions_class()
        self.sequences = {}
        self.tagger_config = {}

    def set_parameters(self, parameters):
        self.parameters.update(parameters)

    def generate_sequence(self, measurement):
        self.sequences = {}
        self.tagger_config = {}

        if hasattr(self.definitions, measurement):
             generator = getattr(self.definitions, measurement)
             # Note: Expects generator to return (sequences, tagger_config, meas_time)
             self.sequences, self.tagger_config, meas_time = generator(self.parameters)
             return meas_time
        else:
            raise ValueError(f"Measurement '{measurement}' is not implemented.")

    def make_pulsestreamer_sequence(self):
        # Helper to convert stored data to PulseStreamer sequence using channel_map
        seq = pulsestreamer.pulse_streamer.createSequence()
        awg_data_map = {}

        for name, pattern in self.sequences.items():
            if name in self.channel_map:
                type, ch = self.channel_map[name]
                if type == 'Analog':
                    seq.setAnalog(ch, pattern)
                elif type == 'Digital':
                    seq.setDigital(ch, pattern)
                elif type == 'AWG':
                    # Check if pattern is a single block (list of tuples) or list of blocks (list of lists)
                    # We assume a block is a list of tuples [(time, val), ...].
                    # If pattern[0] is a list, we assume it's a list of blocks.
                    if not pattern:
                        continue

                    is_multi_step = isinstance(pattern[0], list)

                    if is_multi_step:
                        # List of blocks, implying multiple sequence steps
                        wfs = [AWG_waveform(b) for b in pattern]
                        awg_data_map[ch] = wfs
                    else:
                        # Single block
                        wf = AWG_waveform(pattern)
                        awg_data_map[ch] = [wf] # Store as list for consistent processing
            else:
                print(f"Warning: Sequence '{name}' is not mapped to any channel.")

        if awg_data_map:
            sorted_channels = sorted(awg_data_map.keys())

            # Determine num_waveforms from the first channel
            num_waveforms = len(awg_data_map[sorted_channels[0]])

            waveforms = []
            m1s = []
            m2s = []

            for ch in sorted_channels:
                ch_wfs = awg_data_map[ch]
                if len(ch_wfs) != num_waveforms:
                    raise ValueError(f"Channel {ch} has {len(ch_wfs)} steps, expected {num_waveforms}.")

                if num_waveforms == 1:
                    # Single step: append the array directly (list of arrays)
                    wf = ch_wfs[0]
                    waveforms.append(wf)
                    m1s.append(np.zeros(len(wf)))
                    m2s.append(np.zeros(len(wf)))
                else:
                    # Multi step: append list of arrays (list of lists of arrays)
                    waveforms.append(ch_wfs)
                    m1s.append([np.zeros(len(w)) for w in ch_wfs])
                    m2s.append([np.zeros(len(w)) for w in ch_wfs])

            nreps = [1] * num_waveforms
            trig_waits = [1] * num_waveforms
            jump_tos = [0] * num_waveforms
            goto_states = [0] * num_waveforms
            goto_states[-1] = 1

            AWG_to_send = {
                'waveforms': waveforms,
                'm1s': m1s,
                'm2s': m2s,
                'nreps': nreps,
                'trig_waits': trig_waits,
                'goto_states': goto_states,
                'jump_tos': jump_tos,
                'channels': sorted_channels
            }
            return seq, AWG_to_send
        else:
            return seq

    def make_time_tagger(self):
        tagger = TimeTagger.createTimeTagger()
        counter = TimeTagger.TimeDifferences(tagger=tagger, **self.tagger_config)
        return tagger, counter

In [ ]:
class update:
    @staticmethod
    def _get_curve_names(name, num_curves):
        # name = 'Rabi', 'T1', 'ODMR', 'b_field_fw'
        names = []
        for i in range(num_curves):
            if i == 0:
                suffix = ""
            elif i == 1:
                suffix = "_2nd"
            elif i == 2:
                suffix = "_3rd"
            else:
                suffix = f"_{i+1}th"
            names.append(f"{name}{suffix}")
        return names

    @staticmethod
    def update_plot(analyze_window_start, analyze_window_length, x_data, name, num_curves=4):
        counts = get_data(analyze_window_start, analyze_window_length)
        curve_names = update._get_curve_names(name, num_curves)

        for i, curve_name in enumerate(curve_names):
            if curve_name in NVscan_gui._NV_spectrum_container:
                NVscan_gui._NV_spectrum_container[curve_name].setData(x_data/1e9, counts[i::num_curves])
        return
        # Example:
        # update.update_plot(analyze_window_start, analyze_window_length, mw_length, 'Rabi',num_curves=4)

    @staticmethod
    def clear_plot(name, num_curves=4):
        curve_names = update._get_curve_names(name, num_curves)
        for curve_name in curve_names:
            if curve_name in NVscan_gui._NV_spectrum_container:
                NVscan_gui._NV_spectrum_container[curve_name].clear()
        return


    @staticmethod
    def update_map(name, image_data, **kwargs):
        # Assumes qafm_gui is available in the global scope
        if name in qafm_gui._image_container:
            qafm_gui._image_container[name].setImage(image=image_data, **kwargs)
        return

        # Example:
        # update.update_map('b_field_fw', B_ext, levels=(-2, 2))

In [ ]:
class DataSaver:
    def __init__(self, base_path):
        self.base_path = base_path

    def save(self, data_dict, parameter_dict=None):
        # Create daily folder
        today_str = str(date.today())
        save_dir = os.path.join(self.base_path, today_str)

        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        timestr = time.strftime("%Y%m%d-%H%M%S")

        # Save data arrays
        for name, data in data_dict.items():
            filename = f"{name}_{timestr}.txt"
            file_path = os.path.join(save_dir, filename)
            np.savetxt(file_path, data)

        # Save parameters
        if parameter_dict is not None:
            param_filename = f"parameter_{timestr}.txt"
            param_path = os.path.join(save_dir, param_filename)
            with open(param_path, "w") as f:
                f.write(str(parameter_dict))

# Example usage:
# saving_path = 'C:/Users/Anvil/Measurement Data/QUDI/Scanning_data/F-V10-27/t-MoTe2/device_1/'
# saver = DataSaver(saving_path)
#
# data_to_save = {
#     'B': B_ext,
#     'PL': PL,
#     'Top_gate_list': Top_gate_list,
#     'Bot_gate_list': Bot_gate_list
# }
# para_dict = {"delta_v_top":mod_voltage,"pi_pulse":pi_pulse,"delay_time": delay_time}
# saver.save(data_to_save, para_dict)

In [ ]:
def generate_2d_scan_points(parameters, plot=False):
    # 1. Define Device/Scan Parameters
    angle_deg = parameters.get('angle_deg', 0)  # Set your rotation angle here, default 0
    theta = np.radians(angle_deg)

    scan_length_x = parameters['scan_length_x']
    scan_length_y = parameters['scan_length_y']
    number_of_points_x = parameters['number_of_points_x']
    number_of_points_y = parameters['number_of_points_y']

    start_x = parameters['start_x']
    start_y = parameters['start_y']

    # Following your style for end points (unrotated basis)
    end_x_rel = scan_length_x - (scan_length_x / number_of_points_x)
    end_y_rel = scan_length_y - (scan_length_y / number_of_points_y)

    # 2. Generate the local relative coordinate vectors
    x_points_rel = np.linspace(0, end_x_rel, number_of_points_x)
    y_points_rel = np.linspace(0, end_y_rel, number_of_points_y)

    scan_points = []

    # 3. Nested loops with Rotation Matrix applied
    # x_rot = x*cos - y*sin
    # y_rot = x*sin + y*cos
    for i in range(number_of_points_y):
        for j in range(number_of_points_x):
            # Current unrotated relative point
            xr = x_points_rel[j]
            yr = y_points_rel[i]

            # Apply rotation and shift by start positions
            rot_x = start_x + (xr * np.cos(theta) - yr * np.sin(theta))
            rot_y = start_y + (xr * np.sin(theta) + yr * np.cos(theta))

            new_position = {'X' : rot_x, 'Y': rot_y}
            scan_points.append(new_position)

    # 4. Reshape into your 2D format
    scan_points_2d = np.reshape(scan_points, (number_of_points_y, number_of_points_x))

    if plot:
        check_x = [[p['X'] for p in row] for row in scan_points_2d]
        check_y = [[p['Y'] for p in row] for row in scan_points_2d]
        import matplotlib.pyplot as plt
        plt.figure()
        plt.scatter(check_x, check_y, s=.1)
        plt.axis('equal')
        plt.show()

    return scan_points_2d

# Example Usage:
# scan_params = {
#     'angle_deg': 20,
#     'scan_length_x': 2000e-9,
#     'scan_length_y': 4000e-9,
#     'number_of_points_x': 50,
#     'number_of_points_y': 100,
#     'start_x': 4.78e-6,
#     'start_y': 6.5e-6
# }
# points = generate_2d_scan_points(scan_params, plot=True)

def run_rabi_measurement(mw_freq, mw_power, n_runs, rabi_params, analyze_window_start=0, analyze_window_length=200):
    global counter

    # 1. Define Channel Map (consistent with previous cells)
    channel_map = {
        'laser':('Digital',0),
        'counter':('Digital',5),
        'sync':('Digital',6),
        'I':('Analog',0),
        'Q':('Analog',1)
    }

    # 2. Setup Measurement Sequence
    ms = measurement_sequence(rabi_params, channel_map)
    meas_time = ms.generate_sequence('Rabi')
    seq = ms.make_pulsestreamer_sequence()
    tagger, counter = ms.make_time_tagger()

    # 3. Configure Microwave Source
    # Assuming mw_source_sgs100A is the global object available
    mw_source_sgs100A.set_frequency(mw_freq)
    mw_source_sgs100A.set_power(mw_power)
    mw_source_sgs100A.cw_on()

    # Allow a small delay for equipment to settle
    time.sleep(0.05)

    # 4. Start Measurement
    counter.clear()
    # Using startNow() immediately after streaming
    pulsestreamer.pulse_streamer.stream(seq, n_runs)
    pulsestreamer.pulse_streamer.startNow()

    # 5. Wait loop (Blocking)
    while not pulsestreamer.pulse_streamer.hasFinished():
        time.sleep(0.1)

    # 6. Stop Microwave
    mw_source_sgs100A.off()

    # 7. Retrieve and Process Data
    counts = get_data(analyze_window_start, analyze_window_length)

    # Generate x-axis data (tau)
    taus = np.linspace(rabi_params['tau_start'], rabi_params['tau_end'], rabi_params['num_points'])

    return taus, counts

def fit_Rabi(taus,counts):
    try:
        pulsedmeasurementlogic.fc.set_current_fit('rabi')
        x_fit, y_fit, result = pulsedmeasurementlogic.fc.do_fit(taus/1e9,counts)
        period = 1/result.best_values['frequency']*1e9
        rabi_pi_pulse = period/2
    except:
        rabi_pi_pulse =  mw_time[np.argmin(counts)]
    return rabi_pi_pulse

def

In [ ]:
# Example usage:
# params = {'pi_pulse': 60, 'freq_dev': 1e6, 'num_freq': 101}
# channel_map = {
#     'mw_mod': ('Analog', 0),
#     'mw_switch': ('Digital', 2),
#     'laser': ('Digital', 0),
#     'gate': ('Digital', 5),
#     'sync': ('Digital', 6)
# }
# ms = measurement_sequence(params, channel_map)
# meas_time = ms.generate_sequence('pulsed_ODMR')
# seq = ms.make_pulsestreamer_sequence()
# tagger, counter = ms.make_time_tagger()
# pulsestreamer.pulse_streamer.stream(seq) # Example upload

In [ ]:
# Rabi
params = {
    'laser_time':,
    'wait_time':,
    'tau_start':,
    'tau_end':,
    'num_points':,
    'phi':
    }
taus = np.linspace(params['tau_start'],params['tau_end'],params['num_points'])

channel_map = {
    'laser':('Digital',0),
    'counter':('Digital',5),
    'sync':('Digital',6),
    'I':('Analog',0),
    'Q':('Analog',1)
}

def handle_timeout():
    if pulsestreamer.pulse_streamer.hasFinished():
         timer.stop()
         mw_source_smb100B.off()

    update.update_plot(analyze_window_start, analyze_window_length, taus, 'Rabi',num_curves=1)

ms = measurement_sequence(params, channel_map)
meas_time = ms.generate_sequence('Rabi')
seq = ms.make_pulsestreamer_sequence()
tagger, counter = ms.make_time_tagger()

analyze_window_start = 0
analyze_window_length = 200
integration_time = 100 #sec
n_runs = int(integration_time*1e9/meas_time)

timer = QtCore.QTimer(interval=500, timeout=handle_timeout)
timer.start()


mw_freq = 2.705e9
mw_source_sgs100A.set_frequency(mw_freq)
mw_source_sgs100A.cw_on()
mw_source_sgs100A.set_power(-3)

counter.clear()
time.sleep(0.1)
pulsestreamer.pulse_streamer.stream(seq,n_runs)
pulsestreamer.pulse_streamer.startNow()


In [ ]:
pulsestreamer.pulse_streamer.forceFinal()
timer.stop()
mw_source_sgs100A.off()